In [11]:
import pandas as pd
import csv
import re
import shutil
import os
import numpy as np

import nibabel as nib
import skimage.transform as skTrans
import SimpleITK as sitk

In [12]:
clinical_data_path = 'L:\\basic\diva1\Onderzoekers\DEEP-RISK\All clinical information.csv'
clinical_df = pd.read_csv(clinical_data_path)
clinical_df = clinical_df.copy()
unique_clinical_df = clinical_df.drop_duplicates(subset=['StudyID'], keep='first')
primair_ICM = unique_clinical_df[(unique_clinical_df['ICM'] == 1) & (unique_clinical_df['Implantation_indication'] == 1)]
print(len(primair_ICM))
distant_log_path = 'L:\\basic\diva1\Onderzoekers\DEEP-RISK\parsing_MRI\LOG\\final_header_df_DISTANT.csv'
deeprisk_log_path = 'L:\\basic\diva1\Onderzoekers\DEEP-RISK\parsing_MRI\LOG\\final_header_df_DEEP-RISK.csv'

In [8]:
names = ['Study_MRI_ID', 'Type', 'Series', 'Description', 'Plane', 'StudyDateMRI', 'Age', '?', 'Weight', 9, 10, 11, 12, 13, 14, 15]
dist_log = pd.read_csv(distant_log_path,header=None, names=names, sep=";")

# new data frame with split value columns
temp = dist_log["Study_MRI_ID"].str.split("_", n = 3, expand = True)
# making separate first name column from new data frame
dist_log['StudyID']= temp[1].str.cat(temp[2].astype(str), sep="_")
# making separate last name column from new data frame
dist_log['MRI_ID']= temp[3]
# Dropping old Name columns
# dist_log.drop(columns =["Study_MRI_ID"], inplace = True)
 

dr_log = pd.read_csv(deeprisk_log_path,header=None, names=names, sep=";")

# new data frame with split value columns
temp = dr_log["Study_MRI_ID"].str.split("_", n = 1, expand = True)
# making separate first name column from new data frame
dr_log['StudyID']= temp[0]
# making separate last name column from new data frame
dr_log['MRI_ID']= temp[1]
# Dropping old Name columns
# dr_log.drop(columns =["Study_MRI_ID"], inplace = True)
print(dist_log['StudyID'].nunique())
print(dr_log['StudyID'].nunique())
dr_log.head()


1144
837


,Study_MRI_ID,Type,Series,Description,Plane,StudyDateMRI,Age,?,Weight,9,10,11,12,13,14,15,StudyID,MRI_ID
0,DRAUMC0001_DERI200909030911,CINE,SAX,"tf2d15_retro-triggered_(256,224)",coronal,20090903,065Y,1.75,75.0,NLRAMS0003324815,GR,"['SK', 'SS', 'OSP']",HFS,"[0, 256, 161, 0]","[256, 224, 20, 1]",dicom2nifti,DRAUMC0001,DERI200909030911
1,DRAUMC0001_DERI200909030911,CINE,SAX,"tf2d15_retro-triggered_(256,192)",sagittal,20090903,065Y,1.75,75.0,NLRAMS0003324815,GR,"['SK', 'SS', 'OSP']",HFS,"[0, 256, 138, 0]","[256, 192, 20, 1]",dicom2nifti,DRAUMC0001,DERI200909030911
2,DRAUMC0001_DERI200909030911,CINE,SAX,"tf2d15_retro-triggered_(208,256)",transverse,20090903,065Y,1.75,75.0,NLRAMS0003324815,GR,"['SK', 'SS', 'OSP']",HFS,"[256, 0, 0, 150]","[208, 256, 20, 1]",dicom2nifti,DRAUMC0001,DERI200909030911
3,DRAUMC0001_DERI200909030911,CINE,SAX,"tf2d15_retro-triggered_(224,256)",transverse,20090903,065Y,1.75,75.0,NLRAMS0003324815,GR,"['SK', 'SS', 'OSP']",HFS,"[256, 0, 0, 161]","[224, 256, 20, 1]",dicom2nifti,DRAUMC0001,DERI200909030911
4,DRAUMC0001_DERI200909030911,LGE,LGE_SAX,"ipat_psir_(192,168)",coronal,20090903,065Y,1.75,75.0,NLRAMS0003324815,"['GR', 'IR']","['SK', 'SS', 'MP', 'OSP']",HFS,"[0, 192, 136, 0]","[192, 168, 3, 1]",cardio_nifti,DRAUMC0001,DERI200909030911


In [7]:

# LGE_SAX
mri_type = 'SAX'

# Make a copy of the DataFrame using the copy() method
cine_dist_df = dist_log.loc[dist_log['Series'] == mri_type].copy()
cine_dr_df = dr_log.loc[dr_log['Series'] == mri_type].copy()

# Make unique series
# cine_dist_df['Series'] = cine_dist_df[0]
cine_dist_df.drop_duplicates(subset=['StudyID', 'MRI_ID'], keep='first', inplace=True)
cine_dist_df['StudyDateMRI'] = pd.to_datetime(cine_dist_df['StudyDateMRI'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

# Make unique series
# cine_dr_df['Series'] = cine_dr_df[0]
cine_dr_df.drop_duplicates(subset=['StudyID', 'MRI_ID'], keep='first', inplace=True)
cine_dr_df['StudyDateMRI'] = pd.to_datetime(cine_dr_df['StudyDateMRI'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

print(cine_dist_df.shape)
print(cine_dr_df.shape)



(1291, 18)
(1134, 18)


In [9]:
# LGE_SAX
mri_type = 'LGE_SAX'

# Make a copy of the DataFrame using the copy() method
lge_dist_df = dist_log.loc[dist_log['Series'] == mri_type].copy()
lge_dr_df = dr_log.loc[dr_log['Series'] == mri_type].copy()

# Make unique series
# lge_dist_df['Series'] = lge_dist_df[0]
lge_dist_df.drop_duplicates(subset=['StudyID', 'MRI_ID'], keep='first', inplace=True)
lge_dist_df['StudyDateMRI'] = pd.to_datetime(lge_dist_df['StudyDateMRI'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

# Make unique series
# lge_dr_df['Series'] = lge_dr_df[0]
lge_dr_df.drop_duplicates(subset=['StudyID', 'MRI_ID'], keep='first', inplace=True)
lge_dr_df['StudyDateMRI'] = pd.to_datetime(lge_dr_df['StudyDateMRI'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

print(lge_dist_df.shape)
print(lge_dr_df.shape)

(1201, 18)
(939, 18)


In [10]:
clinical_df = pd.read_csv(clinical_data_path)
implantdetails = clinical_df[['StudyID', 'ImplantationDate']]

lge_dr__ = lge_dr_df[['StudyID', 'StudyDateMRI', 'MRI_ID', 'Study_MRI_ID']]
lge_dist__ = lge_dist_df[['StudyID', 'StudyDateMRI',  'MRI_ID', 'Study_MRI_ID']]

print(lge_dr__.shape)
print(lge_dist__.shape)
lge_mri = pd.concat([lge_dist__, lge_dr__], axis=0)

# lge_mri['Filename'] = lge_mri[0]
# print(lge_mri.shape)
print(lge_mri.shape)
print(len(lge_mri['StudyID'].unique()))
lge_mri.head()



NameError: name 'lge_dr_df' is not defined

In [24]:
clinical_df = pd.read_csv(clinical_data_path)
clinical_df = clinical_df.copy()
unique_clinical_df = clinical_df.drop_duplicates(subset=['StudyID'], keep='first')
primair_ICM = unique_clinical_df[(unique_clinical_df['ICM'] == 1) & (unique_clinical_df['Implantation_indication'] == 1)]
print(len(primair_ICM))


749


In [12]:
# cine_dr__ = cine_dr_df[['StudyID', 'StudyDateMRI','MRI_ID', 'Study_MRI_ID']]
# cine_dist__ = cine_dist_df[['StudyID', 'StudyDateMRI',  'MRI_ID', 'Study_MRI_ID']]

# print(cine_dr__.shape)
# print(cine_dist__.shape)
# cine_mri = pd.concat([cine_dist__, cine_dr__], axis=0)

# # cine_mri['Filename'] = cine_mri['StudyID']
# print(cine_mri.shape)
# print(len(cine_mri['StudyID'].unique()))
# cine_mri.head()

(1134, 4)
(1291, 4)
(2425, 4)
1847


,StudyID,StudyDateMRI,MRI_ID,Study_MRI_ID
0,DIST_0927,2006-08-24,A1207,20060824_DIST_0927_A1207
3,DIST_0888,2006-09-25,A0890,20060925_DIST_0888_A0890
8,DIST_0046,2006-11-06,A1256,20061106_DIST_0046_A1256
11,DIST_2951,2006-11-06,A0323,20061106_DIST_2951_A0323
14,DIST_2148,2006-11-20,A0955,20061120_DIST_2148_A0955


In [13]:
# cine_lge_mri = pd.merge(lge_mri, cine_mri)
# print(cine_lge_mri.shape)
# print(len(cine_lge_mri['StudyID'].unique()))
# cine_lge_mri.head()

(1973, 4)
1608


,StudyID,StudyDateMRI,MRI_ID,Study_MRI_ID
0,DIST_0668,2006-11-23,A0044,20061123_DIST_0668_A0044
1,DIST_3407,2007-02-12,A1371,20070212_DIST_3407_A1371
2,DIST_0312,2007-05-03,A0373,20070503_DIST_0312_A0373
3,DIST_0251,2007-05-14,A0762,20070514_DIST_0251_A0762
4,DIST_1073,2007-05-14,A0046,20070514_DIST_1073_A0046


In [11]:
clin_mri = pd.merge(lge_mri, implantdetails, on='StudyID', how='left')

print(clin_mri.shape)
clin_mri = clin_mri[['StudyID', 'StudyDateMRI','ImplantationDate', 'Study_MRI_ID']]

#Calculate days between implant and MRI
clin_mri['StudyDateMRI'] = pd.to_datetime(clin_mri['StudyDateMRI'])
clin_mri['ImplantationDate'] = pd.to_datetime(clin_mri['ImplantationDate'])

# Calculate the time difference in days between 'StudyDateMRI' and 'ImplantationDate'.
clin_mri['Days_MRI_to_Implant'] = (clin_mri['StudyDateMRI'] - clin_mri['ImplantationDate']).dt.days

#Baseline MRI
#indices = clin_mri[(clin_mri['Days_MRI_to_Implant'] <0) & (clin_mri['Days_MRI_to_Implant'] >-365)].index.tolist()
#clin_mri = clin_mri[clin_mri.index.isin(indices)]

clin_mri_onemriperpatient = clin_mri

#Select so one row per patient, keep MRI closest to implant
clin_mri_onemriperpatient = clin_mri_onemriperpatient.sort_values(by='Days_MRI_to_Implant')
clin_mri_onemriperpatient = clin_mri_onemriperpatient.drop_duplicates(subset='StudyID', keep='last')

print("number of MRI total", clin_mri.shape)
print("number of patients with baseline MRI", (clin_mri_onemriperpatient['StudyID'].nunique()))

subject_counts = clin_mri.groupby('StudyID').size()

# Count the occurrences of each unique value in the subject_counts Series
subject_counts_summary = subject_counts.value_counts()

print("Number of subjects with 1 MRI:", subject_counts_summary.get(1, 0))
print("Number of subjects with 2 MRI:", subject_counts_summary.get(2, 0))
print("Number of subjects with 3 MRI:", subject_counts_summary.get(3, 0))
print("Number of subjects with 4 MRI:", subject_counts_summary.get(4, 0))
print("Number of subjects with 5 MRI:", subject_counts_summary.get(5, 0))
print("Number of subjects with 5 MRI:", subject_counts_summary.get(6, 0))

(2140, 5)
number of MRI total (2140, 5)
number of patients with baseline MRI 1722
Number of subjects with 1 MRI: 1432
Number of subjects with 2 MRI: 210
Number of subjects with 3 MRI: 55
Number of subjects with 4 MRI: 16
Number of subjects with 5 MRI: 5
Number of subjects with 5 MRI: 0


In [13]:
lge_logmri_path = 'L:\\basic\\Personal Archive\\E\\emquist\\parsing_MRI\\final_test_seg\\LOG\\lge_sax_psir_log_good_shape.csv'
# cine_logmri_path = 'L:\\basic\\Personal Archive\\E\\emquist\\parsing_MRI\\final_test_seg\\LOG\\cine_sax_log_good_shape.csv'

In [16]:
# old_lge_logmri_path = 'L:\\basic\\Personal Archive\\E\\emquist\\parsing_MRI\\test_mri_seg\\LOG\\log_good.csv'
# old_cine_logmri_path = 'L:\\basic\\Personal Archive\\E\\emquist\\parsing_MRI\\test_mri_seg\\LOG\\cine_log_good.csv'

In [35]:

lge_mri_good_df = pd.read_csv(lge_logmri_path, header=0, sep=',')
# print(lge_mri_good_df)
# Split the data paths column on '/'
# lge_mri_good_df['Processed_Paths'] = lge_mri_good_df['Processed_Paths'].replace(" ", "_", regex=True).str.strip()
split_path= lge_mri_good_df['Processed_Paths'].str.split('\\', expand=True)

# Assign the split values to new columns
for i, col in enumerate(split_path.columns):
    lge_mri_good_df[col] = split_path[i]

columns_to_drop = [i for i in range(8)]
# columns_to_drop = [i for i in range(11)]

lge_mri_good_df = lge_mri_good_df.drop(columns=columns_to_drop, axis=1)

lge_mri_good_df = lge_mri_good_df.drop(columns=[9, 'Slices'], axis=1)
# lge_mri_good_df = lge_mri_good_df.drop(columns=[11, 13], axis=1)

lge_mri_good_df.rename(columns={'Processed_Paths':'LGE_paths', 'Shape': 'LGE_shape', 8: 'Patient_MRI_ID', 10: 'LGE_filename', 'number': 'LGE_number'}, inplace=True)
# lge_mri_good_df.rename(columns={'Processed_Paths':'LGE_paths', 'Slices': 'LGE_slices', 'Shape': 'LGE_shape', 12: 'Patient_MRI_ID', 14: 'LGE_filename'}, inplace=True)

lge_split_id = lge_mri_good_df['Patient_MRI_ID'].str.split('_', expand=True)

# Split the DataFrame based on condition (Deep Risk or Distant)

lge_DR_split_id = lge_split_id[lge_split_id[0].astype(str).str.startswith('D')]

lge_DR_split_id = lge_DR_split_id.drop(columns=[2, 3], axis=1)
lge_DR_split_id.rename(columns={0: 'StudyID', 1: 'MRI_ID'}, inplace=True)

lge_DIST_split_id = lge_split_id[~lge_split_id[0].astype(str).str.startswith('D')].copy()
lge_DIST_split_id['StudyID']= lge_DIST_split_id[1].str.cat(lge_DIST_split_id[2].astype(str), sep="_")
lge_DIST_split_id['MRI_ID'] = lge_DIST_split_id[0].str.cat(lge_DIST_split_id[3].astype(str), sep="_")
lge_DIST_split_id = lge_DIST_split_id.drop(columns=[0, 1, 2, 3], axis=1)

# Merge the split DataFrames and restore the original index order
merged_split_id = pd.concat([lge_DR_split_id, lge_DIST_split_id]).sort_index()
lge_mri_good_df = pd.merge(lge_mri_good_df, merged_split_id, left_index=True, right_index=True)
lge_mri_good_df = lge_mri_good_df.drop(columns=['Patient_MRI_ID'], axis=1)
# print(lge_mri_good_df['Patient_MRI_ID'].nunique())
print(len(lge_mri_good_df))
lge_mri_good_df.head()


1078


,LGE_paths,LGE_shape,LGE_number,LGE_filename,StudyID,MRI_ID
0,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(174, 192, 10)",0,"sagittal_tfi_psir_single-shot_10sl_psir_(192,1...",DIST_0312,20070503_A0373
1,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(144, 192, 10)",1,"sagittal_tfi_psir_single-shot_10sl_psir_(192,1...",DIST_1869,20080130_A0365
2,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(144, 192, 10)",2,"sagittal_tfi_psir_single-shot_10sl_psir_(192,1...",DIST_2446,20080306_A0933
3,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(192, 144, 10)",3,"coronal_tfi_psir_single-shot_10sl_psir_(144,19...",DIST_2249,20080417_A0243
4,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(144, 192, 10)",4,"sagittal_tfi_psir_single-shot_10sl_psir_(192,1...",DIST_2514,20080731_A1248


In [22]:
# cine_mri_good_df = pd.read_csv(cine_logmri_path, header=0, sep=',')
# cine_mri_good_df = cine_mri_good_df.drop(index=0)

# # Split the data paths column on '/'
# # cine_mri_good_df['Processed_Paths'] = cine_mri_good_df['Processed_Paths'].replace(" ", "_", regex=True).str.strip()
# split_data = cine_mri_good_df['Processed_Paths'].str.split('\\', expand=True)

# # Assign the split values to new columns
# for i, col in enumerate(split_data.columns):
#     cine_mri_good_df[col] = split_data[i]

# columns_to_drop = [i for i in range(8)]
# # columns_to_drop = [i for i in range(11)]
# cine_mri_good_df = cine_mri_good_df.drop(columns=columns_to_drop, axis=1)
# cine_mri_good_df = cine_mri_good_df.drop(cine_mri_good_df.index[0])

# cine_mri_good_df = cine_mri_good_df.drop(columns=[9, 'Slices'], axis=1)
# # cine_mri_good_df = cine_mri_good_df.drop(columns=[11], axis=1)

# cine_mri_good_df.rename(columns={'Processed_Paths':'CINE_paths', 'Shape': 'CINE_shape', 8: 'Patient_MRI_ID', 10: 'CINE_filename', 'number': 'CINE_number'}, inplace=True)
# # cine_mri_good_df.rename(columns={'Processed_Paths':'CINE_paths', 'Slices': 'CINE_slices', 'Shape': 'CINE_shape', 12: 'Patient_MRI_ID', 14: 'CINE_filename'}, inplace=True)

# cine_split_id = cine_mri_good_df['Patient_MRI_ID'].str.split('_', expand=True)
# # Split the DataFrame based on condition (Deep Risk or Distant)
# cine_DR_split_id = cine_split_id[cine_split_id[0].astype(str).str.startswith('D')]
# cine_DR_split_id = cine_DR_split_id.drop(columns=[2, 3], axis=1)
# cine_DR_split_id.rename(columns={0: 'StudyID', 1: 'MRI_ID'}, inplace=True)

# cine_DIST_split_id = cine_split_id[~cine_split_id[0].astype(str).str.startswith('D')].copy()
# cine_DIST_split_id['StudyID']= cine_DIST_split_id[1].str.cat(cine_DIST_split_id[2].astype(str), sep="_")
# cine_DIST_split_id['MRI_ID'] = cine_DIST_split_id[0].str.cat(cine_DIST_split_id[3].astype(str), sep="_")
# cine_DIST_split_id = cine_DIST_split_id.drop(columns=[0, 1, 2, 3], axis=1)

# # Merge the split DataFrames and restore the original index order
# cine_merged_split_id = pd.concat([cine_DR_split_id, cine_DIST_split_id]).sort_index()
# cine_mri_good_df = pd.merge(cine_mri_good_df, cine_merged_split_id, left_index=True, right_index=True)
# cine_mri_good_df = cine_mri_good_df.drop(columns=['Patient_MRI_ID'], axis=1)

# cine_mri_good_df.head()

,CINE_paths,CINE_shape,CINE_number,Patient_MRI_ID,CINE_filename,StudyID,MRI_ID
2,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(192, 192, 12, 25)",2,20070712_DIST_0811_A1075,"coronal_tf2d22_retro_ipat-14sl_(192,192).nii.gz",DIST_0811,20070712_A1075
3,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(208, 256, 11, 20)",3,20070909_DIST_0089_A0752,"coronal_tf2d15_retro-triggered_(256,208).nii.gz",DIST_0089,20070909_A0752
4,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(192, 138, 18, 25)",4,20070913_DIST_0393_A0915,"transverse_tf2d22_retro_ipat-14sl_(138,192).ni...",DIST_0393,20070913_A0915
5,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(126, 192, 14, 25)",5,20070917_DIST_1235_A0348,"coronal_tf2d22_retro_ipat-sa(14sl)_(192,126).n...",DIST_1235,20070917_A0348
6,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(208, 256, 14, 20)",6,20071021_DIST_1634_A0990,"coronal_tf2d15_retro-triggered_(256,208).nii.gz",DIST_1634,20071021_A0990


In [23]:
# cine_mri_good_df['Patient_MRI_ID'].nunique()

1392

In [25]:
# cine_lge_mri = pd.merge(lge_mri_good_df, cine_mri_good_df, on=['StudyID', 'MRI_ID'])
# print(cine_lge_mri['StudyID'].nunique())

# csv_filename = 'L:\\basic\\Personal Archive\\E\\emquist\\parsing_MRI\\final_test_seg\\LOG\\merged_lge_cine.csv'
# cine_lge_mri.to_csv(csv_filename, index=False)

# import ast
# # # Convert the string tuples to actual tuples
# cine_lge_mri['CINE_shape'] = cine_lge_mri['CINE_shape'].apply(ast.literal_eval)

# # Extract the third values and calculate the mean
# third_values = [tup[2] for tup in cine_lge_mri['CINE_shape']]
# mean_third_value = sum(third_values) / len(third_values)
# print(mean_third_value)

NameError: name 'cine_mri_good_df' is not defined

In [34]:
clinical_data_path = r"\\amc.intra\opslag\basic\diva1\Onderzoekers\DEEP-RISK\All clinical information.csv"
clinical_df = pd.read_csv(clinical_data_path)
print(len(clinical_df))
clinical_df.head()

2942


,Unnamed: 0,StudyID_x,Python_ID_Index,Age,Sex,AtrialArrhythmia,LowLVEF,ReducedLVEF,NormalLVEF,OHCA,...,ICDnonbenefit_4Y,ICDnonbenefit_3Y,ICDnonbenefit_2Y,ICDnonbenefit_1Y,FU_1Y,FU_2Y,FU_3Y,FU_4Y,FU_5Y,StudyID
0,0,DIST_2789,0,65.322382,1,0,1,0,0,0,...,0,0,0,0,1,1,1,1,1,DIST_2789
1,1,DIST_0815,1,61.125257,1,0,1,0,0,0,...,0,0,0,0,1,1,1,1,1,DIST_0815
2,2,DIST_3632,2,61.045859,0,0,0,0,1,0,...,0,0,0,0,1,1,1,1,0,DIST_3632
3,3,DIST_0517,3,73.305955,1,0,0,0,1,1,...,0,0,0,0,1,1,1,1,1,DIST_0517
4,4,DIST_0461,4,83.006160,1,0,1,0,0,0,...,0,0,0,0,1,1,1,1,1,DIST_0461


In [218]:
# clin_cine = pd.merge(cine_mri_good_df, clinical_df, on=['StudyID'])
# print(len(clin_cine))
# clin_cine['MRI_ID'].nunique()
# csv_filename = 'L:\\basic\\Personal Archive\\E\\emquist\\parsing_MRI\\final_test_seg\\LOG\\clin_cine.csv'
# clin_cine.to_csv(csv_filename, index=False)

1313


In [37]:
clin_lge = pd.merge(lge_mri_good_df, clinical_df, on=['StudyID'])
print(len(clin_lge))
print(clin_lge['MRI_ID'].nunique())
csv_filename = 'L:\\basic\\Personal Archive\\E\\emquist\\parsing_MRI\\final_test_seg\\LOG\\clin_merged_lge.csv'
clin_lge.to_csv(csv_filename, index=False)
clin_lge.head(n=1000)

988
879


,LGE_paths,LGE_shape,LGE_number,LGE_filename,StudyID,MRI_ID,Unnamed: 0,StudyID_x,Python_ID_Index,Age,...,ICDnonbenefit_5Y,ICDnonbenefit_4Y,ICDnonbenefit_3Y,ICDnonbenefit_2Y,ICDnonbenefit_1Y,FU_1Y,FU_2Y,FU_3Y,FU_4Y,FU_5Y
0,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(192, 144, 10)",7,transverse_tfi_psir_single-shot_10sl_ipat_sa_p...,DIST_3815,20090129_A0597,629,DIST_3815,629,54.401095,...,0,0,0,0,0,1,1,1,1,1
1,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(192, 144, 10)",8,coronal_tfi_psir_single-shot_10sl_ipat_sa_psir...,DIST_2327,20090319_A1188,81,DIST_2327,81,61.138946,...,1,1,1,1,1,0,0,0,0,0
2,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(144, 192, 10)",11,sagittal_tfi_psir_single-shot_10sl_ipat_sa_psi...,DIST_1598,20090416_A0701,474,DIST_1598,474,44.057495,...,0,0,0,0,0,1,1,1,1,1
3,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(192, 144, 9)",24,transverse_tfi_psir_single-shot_10sl_ipat_sa_p...,DIST_2932,20091001_A0592,276,DIST_2932,276,46.562628,...,0,0,0,0,0,1,1,1,1,1
4,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(192, 144, 10)",32,coronal_tfi_psir_single-shot_10sl_ipat_sa_psir...,DIST_1427,20100121_A0949,584,DIST_1427,584,31.331964,...,0,0,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(170, 192, 10)",1073,sagittal_de_overview_single_shot_sax_psir_(192...,DRAUMC1252,DERI202004291135,2930,NaN,2930,65.338809,...,0,0,0,0,0,0,0,0,0,0
984,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(192, 192, 11)",1074,"coronal_de_overview_single_shot_sax_psir_(192,...",DRAUMC1254,DERI202004290847,2932,NaN,2932,54.387406,...,0,0,0,0,0,0,0,0,0,0
985,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(162, 192, 12)",1075,transverse_de_overview_single_shot_sax_psir_(1...,DRAUMC1256,DERI202005251429,2933,NaN,2933,77.407255,...,0,0,0,0,0,0,0,0,0,0
986,L:\basic\Personal Archive\E\emquist\parsing_MR...,"(192, 192, 12)",1076,"coronal_lge_single-shot_sax_psir_(192,192).nii.gz",DRAUMC1258,DERI202006041554,2935,NaN,2935,53.445585,...,0,0,0,0,0,0,0,0,0,0


In [39]:
lge_for_clin = 'L:\\basic\\Personal Archive\\E\emquist\\parsing_MRI\\final_test_seg\\lge_for_clin'
if not os.path.exists(lge_for_clin):
    os.makedirs(lge_for_clin, exist_ok=True)
lge_og_path = 'L:\\basic\Personal Archive\E\emquist\parsing_MRI\\final_test_seg\lge_sax_psir_mri_good_shape'

# cine_for_merged = 'L:\\basic\\Personal Archive\\E\emquist\\parsing_MRI\\final_test_seg\\cine_for_merged_clin'
# if not os.path.exists(cine_for_merged):
#     os.makedirs(cine_for_merged, exist_ok=True)
# cine_og_path = 'L:\\basic\Personal Archive\E\emquist\parsing_MRI\\final_test_seg\cine_sax_mri_good_shape'

for index, row in clin_lge.iterrows():
    full_lge_og_path = os.path.join(lge_og_path, str(row['LGE_number']) + '_' + row['LGE_filename'])
    # full_cine_og_path = os.path.join(cine_og_path, str(row['CINE_number']) + '_' + row['CINE_filename'])
    lge_new_path = os.path.join(lge_for_clin, str(index) + '_' + row['StudyID'] + '_' + row['MRI_ID'] +'_' + row['LGE_filename'])
    # cine_new_path = os.path.join(cine_for_merged, str(index) + '_' + row['StudyID'] + '_' + row['MRI_ID'] +'_' + row['CINE_filename'])
    shutil.copy2(full_lge_og_path, lge_new_path)
    # shutil.copy2(full_cine_og_path, cine_new_path)

There are 409 healthy and 322 MyocardialInfarction patients in the dataset.
DRAUMC1112_DERI201810311130 with MINF
Image path L:\basic\Personal Archive\E\emquist\parsing_MRI\final_test_seg\lge_for_merged_clin\855_DRAUMC1112_DERI201810311130_coronal_lge_single-shot_11sl_ipat_psir_(192,192).nii.gz


Image/Mask geometry mismatch, attempting to correct Mask


ValueError: ('Label (%d) not present in mask', 1)